In [1]:
import tensorflow as tf
import json
from collections import defaultdict

#from khaiii import KhaiiiApi
#api = KhaiiiApi()

import MeCab
api = MeCab.Tagger('-d /usr/local/lib/mecab/dic/mecab-ko-dic')


In [2]:
def tokenize(anal):
    
    toks = []
    
    for word in anal:
        for morp in str(word).split('\t')[1].split('+'):
            toks.append(morp.strip())
    
    return toks


In [3]:
def tokenize2(text):

    toks = []

    for word in api.parse(text).split('\n'):
        if word == 'EOS':
            break
        mophs = word.split('\t')
        els = mophs[1].split(',')
        if els[7] == '*':
            toks.append(mophs[0] + '/' + els[0])
        else:
            for dels in els[7].split('+'):
                dmophs = dels.split('/')
                toks.append(dmophs[0] + '/' + dmophs[1])
                
    return toks


In [4]:
def read_text(dic, text):
    
    for token in tokenize2(text.strip()):
        
        token = token.strip()
        
        dic_ori[token] += 1


In [5]:
def apply_mincut(frequency_ori, min_freq):
    
    print('apply minCut and re-generate minCutDic')
    mincut_dic = dict(filter(lambda args : args[1] > min_freq, frequency_ori.items()))

    return mincut_dic


In [6]:
dic_ori = defaultdict(int)

num_entry = 0
num_context = 0
num_question = 0

data_set = []
entry_set = []

with tf.gfile.Open('./KorQuAD_v1.0_train.json', "r") as reader:
    input_data = json.load(reader)["data"]
    
    for entry in input_data:
        num_entry = num_entry + 1
        for paragraph in entry["paragraphs"]:
            num_context = num_context + 1
            paragraph_text = paragraph["context"]
            
            read_text(dic_ori, paragraph_text)
            
            list_ques = []
            
            for qa in paragraph["qas"]:
                num_question = num_question + 1
                qas_id = qa["id"]
                question_text = qa["question"]
                
                list_ques.append(question_text)
                
                read_text(dic_ori, question_text)
                
                answer = qa["answers"][0]
                orig_answer_text = answer["text"]
                answer_offset = answer["answer_start"]
                
            data_set.append( [paragraph_text, list_ques] )
            entry_set.append( num_entry - 1 )


In [7]:
print(len(dic_ori))
print(num_entry)
print(num_context)
print(num_question)


69529
1420
9681
60407


In [8]:
print(paragraph_text)

2010년 8월, 저널 The Lancet Infectious Diseases에 최근 행해진 다국적 연구 결과가 실린 바 있다. 이 연구는 NDM-1 유전자를 가진 박테리아의 발생과 전파를 분석하였다. 연구에 사용된 케이스는 영국 37건, 첸나이 26건, 인도 하리아나 주 26건, 그 외 인도, 파키스탄의 여러 지역 73건 등이었다. 저자의 말에 따르면, 많은 균주들이 NDM-1 유전자를 플라스미드 상에 가지고 있었으며, 이 때문에 균과 균 사이의 유전자 전달 (gene transfer)이 쉽게 가능할 것으로 보았다. 연구 시 다뤄진 모든 균주는 베타-락탐계열 항생제, 퀴놀론 (quinolone) 계열 항생제, 아미노글리코사이드 등 여러 항생제에 내성을 보였으나, 대부분 폴리믹신 (polymyxin) 계열 항생제 콜리스틴 (colistin)에는 감수성을 보였다.


In [9]:
print(tokenize2(paragraph_text.strip()))

['2010/SN', '년/NNBC', '8/SN', '월/NNBC', ',/SC', '저널/NNG', 'The/SL', 'Lancet/SL', 'Infectious/SL', 'Diseases/SL', '에/JKB', '최근/NNG', '행하/VV', '아/EC', '지/VX', 'ᆫ/ETM', '다/NNG', '국적/NNG', '연구/NNG', '결과/NNG', '가/JKS', '실리/VV', 'ᆫ/ETM', '바/NNB', '있/VV', '다/EF', './SF', '이/MM', '연구/NNG', '는/JX', 'NDM/SL', '-/SY', '1/SN', '유전/NNG', '자/NNG', '를/JKO', '가지/VV', 'ᆫ/ETM', '박테리아/NNG', '의/JKG', '발생/NNG', '과/JC', '전파/NNG', '를/JKO', '분석/NNG', '하/XSV', '였/EP', '다/EF', './SF', '연구/NNG', '에/JKB', '사용/NNG', '되/XSV', 'ᆫ/ETM', '케이스/NNG', '는/JX', '영국/NNP', '37/SN', '건/NNBC', ',/SC', '첸나이/NNP', '26/SN', '건/NNBC', ',/SC', '인도/NNP', '하리아나/NNP', '주/NNG', '26/SN', '건/NNBC', ',/SC', '그/MM', '외/NNB', '인도/NNP', ',/SC', '파키스탄/NNP', '의/JKG', '여러/MM', '지역/NNG', '73/SN', '건/NNBC', '등/NNB', '이/VCP', '었/EP', '다/EF', './SF', '저자/NNG', '의/JKG', '말/NNG', '에/JKB', '따르/VV', '면/EC', ',/SC', '많/VA', '은/ETM', '균주/NNG', '들/XSN', '이/JKS', 'NDM/SL', '-/SY', '1/SN', '유전/NNG', '자/NNG', '를/JKO', '플라스미드/NNG', '상/NNG', '에/JKB', '가지/VV', 

In [10]:
print(len(entry_set))

9681


In [11]:
num_entry = 0
num_context = 0
num_question = 0

with tf.gfile.Open('./ko_wiki_v1_squad.json', "r") as reader:
    input_data = json.load(reader)["data"]
    
    for entry in input_data:
        num_entry = num_entry + 1
        for paragraph in entry["paragraphs"]:
            num_context = num_context + 1
            paragraph_text = paragraph["context"]
            
            read_text(dic_ori, paragraph_text)
            
            list_ques = []
            
            for qa in paragraph["qas"]:
                num_question = num_question + 1
                qas_id = qa["id"]
                question_text = qa["question"]
                
                list_ques.append(question_text)
                
                read_text(dic_ori, question_text)
                
                answer = qa["answers"][0]
                orig_answer_text = answer["text"]
                answer_offset = answer["answer_start"]
                
            data_set.append( [paragraph_text, list_ques] )


In [12]:
print(len(dic_ori))
print(num_entry)
print(num_context)
print(num_question)


148261
68538
68538
100268


In [13]:
num_entry = 0
num_context = 0
num_question = 0

data_set_val = []

with tf.gfile.Open('./KorQuAD_v1.0_dev.json', "r") as reader:
    input_data = json.load(reader)["data"]
    
    for entry in input_data:
        num_entry = num_entry + 1
        for paragraph in entry["paragraphs"]:
            num_context = num_context + 1
            paragraph_text = paragraph["context"]
            
            read_text(dic_ori, paragraph_text)
            
            list_ques = []
            
            for qa in paragraph["qas"]:
                num_question = num_question + 1
                qas_id = qa["id"]
                question_text = qa["question"]
                
                list_ques.append(question_text)
                
                read_text(dic_ori, question_text)
                
                answer = qa["answers"][0]
                orig_answer_text = answer["text"]
                answer_offset = answer["answer_start"]
                
            data_set_val.append( [paragraph_text, list_ques] )


In [14]:
print(len(dic_ori))
print(num_entry)
print(num_context)
print(num_question)


149722
140
964
5774


In [15]:
#dic = apply_mincut(dic_ori, 3)
dic = dic_ori
print(len(dic))


149722


In [16]:
voca = {}

voca['_PAD_'] = len(voca)
voca['_UNK_'] = len(voca)

for word in dic.keys():
    voca[word] = len(voca)    
    
print(len(voca))


149724


In [17]:
import pickle
pickle.dump( voca, open('./v1_dic.pkl', 'wb')  )


In [18]:
dic = pickle.load(open('./v1_dic.pkl', 'rb') )
print(len(dic))


149724


In [19]:
num_dt = 0
num_dt2 = 0

for data in data_set:
    num_dt = num_dt + 1
    for data2 in data[1]:
        num_dt2 = num_dt2 + 1

print(num_dt)
print(num_dt2)


78219
160675


In [20]:
num_dt = 0
num_dt2 = 0

for data in data_set_val:
    num_dt = num_dt + 1
    for data2 in data[1]:
        num_dt2 = num_dt2 + 1

print(num_dt)
print(num_dt2)


964
5774


In [21]:
print(data_set[0])

['1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.', ['바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?', '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?', '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?', '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?', '바그너가 파우스트를 처음으로 읽은 년도는?', '바그너가 처음 교향곡 작곡을 한 장소는?', '바그너의 1악장의 초연은 어디서 연주되었는가?']]


In [22]:
print(data_set[0][0])

1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [23]:
print(data_set[0][1])

['바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?', '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?', '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?', '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?', '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?', '바그너가 파우스트를 처음으로 읽은 년도는?', '바그너가 처음 교향곡 작곡을 한 장소는?', '바그너의 1악장의 초연은 어디서 연주되었는가?']


In [24]:
print(data_set[78218])

['계속해서 포기하지 않고 싸우던 미후네 돌핀스는 마침내 요코하마 리틀에게서 승리를 쟁취하지만 승리함과 동시에 기권하였기 때문에 전국대회는 나가지 못한다. 계절의 끝, 고로의 의붓 엄마가 시게노 히데키와 결혼하고 시게노가 마린스타즈에서 이적했기 때문에 새로운 가족은 요코하마를 떠나게 되었다. 그리고 고로는 팀원들에게 언젠가는 돌아오겠다는 말만 남긴채 후쿠오카로 가게 된다. 그의 양아버지가 다시 블루오션스로 트레이드되었을 때 다시 미후네로 돌아오고 리틀 야구 시절 친구들이 자라나던 미후네 동중학교로 전학했다. 그러나, 그가 축구부에 들어가서 축구를 하고 야구부에 들어가서 야구를 하지 않자 친구들은 놀라워하지만 나중에 후쿠오카 리틀리그에서 오른쪽 어깨에 부상을 입은 뒤로 야구부에 들어가지 않았다는 사실을 알고 이해한다. 고로는 야구부를 살리려는 코모리를 위협하는 불량 부원의 마음을 바꿔 놓고 함께 중학교 야구부를 재건한다. 팀은 지역 토너먼트에서 강한 실력을 발휘하고 4강에서 친구이자 라이벌은 사토 토시야가 있는 토모노우라 중학교 야구부를 상대로 승리하는 등 지방대회에서 우승하게 된다.', ['메이저에서 고로가 미후네로 돌아와서 야구부가 아닌 축구부에 들어간 이유가 뭐야']]


In [25]:
print(data_set[78218][0])

계속해서 포기하지 않고 싸우던 미후네 돌핀스는 마침내 요코하마 리틀에게서 승리를 쟁취하지만 승리함과 동시에 기권하였기 때문에 전국대회는 나가지 못한다. 계절의 끝, 고로의 의붓 엄마가 시게노 히데키와 결혼하고 시게노가 마린스타즈에서 이적했기 때문에 새로운 가족은 요코하마를 떠나게 되었다. 그리고 고로는 팀원들에게 언젠가는 돌아오겠다는 말만 남긴채 후쿠오카로 가게 된다. 그의 양아버지가 다시 블루오션스로 트레이드되었을 때 다시 미후네로 돌아오고 리틀 야구 시절 친구들이 자라나던 미후네 동중학교로 전학했다. 그러나, 그가 축구부에 들어가서 축구를 하고 야구부에 들어가서 야구를 하지 않자 친구들은 놀라워하지만 나중에 후쿠오카 리틀리그에서 오른쪽 어깨에 부상을 입은 뒤로 야구부에 들어가지 않았다는 사실을 알고 이해한다. 고로는 야구부를 살리려는 코모리를 위협하는 불량 부원의 마음을 바꿔 놓고 함께 중학교 야구부를 재건한다. 팀은 지역 토너먼트에서 강한 실력을 발휘하고 4강에서 친구이자 라이벌은 사토 토시야가 있는 토모노우라 중학교 야구부를 상대로 승리하는 등 지방대회에서 우승하게 된다.


In [26]:
print(data_set[78218][1])

['메이저에서 고로가 미후네로 돌아와서 야구부가 아닌 축구부에 들어간 이유가 뭐야']


In [27]:
print(data_set_val[0])

['1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.', ['임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?', '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?', '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?', '임종석을 검거한 장소는 경희대 내 어디인가?', '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?', '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?', '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?']]


In [28]:
print(data_set_val[0][0])

1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의(폭력행위등처벌에관한법률위반)으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일~20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다.


In [29]:
print(data_set_val[0][1])

['임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?', '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?', '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?', '임종석을 검거한 장소는 경희대 내 어디인가?', '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?', '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?', '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?']


In [30]:
def find_index(word, dic):
    
    word = word.strip()
    
    if word in dic:
        return dic[word]
    else:
        return dic['_UNK_']


In [31]:
def return_anti(cur, shuff, max_cnt):
    
    data_anti = []
    num_anti = 0
    
    for idx in shuff:
        if idx == cur:
            continue
        for res in data_set[idx][1]:
            num_anti = num_anti + 1
            data_anti.append(res)
            
            if num_anti == max_cnt:
                return data_anti


In [32]:
def return_anti2(cur, shuff):
    
    data_anti = []
    num_anti = 0
    
    for idx in shuff:
        if idx == cur:
            continue
        for res in data_set_val[idx][1]:
            num_anti = num_anti + 1
            data_anti.append(res)
            
            if num_anti == 10:
                return data_anti


In [34]:
from random import shuffle

train_set = {}
context = []
response = []
label = []

cnt = 0
for idx in range(len(data_set)):
    
    num_res = 0
    
    for data_res in data_set[idx][1]:
        response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
        num_res = num_res + 1
    
    if idx < 9681:
        
        cores_x = [i for i, e in enumerate(entry_set) if e == entry_set[idx]]
        
        cores_x.remove(idx)
        
        if len(cores_x) == 0:
            
            rand_x = [i for i in range(len(data_set))]
            
            shuffle(rand_x)
            
            data_anti = return_anti(idx, rand_x, 10)
            
            for data_res in data_anti:
                response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
                
        else:
            
            num_cores_x = 0
            
            for xx in cores_x:
                num_cores_x = num_cores_x + len(data_set[xx][1])
                
            if num_cores_x < 5:
                '''
                shuffle(cores_x)
                
                data_anti = return_anti(idx, cores_x, num_cores_x)
                
                for data_res in data_anti:
                    response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
                '''
                rand_x = [i for i in range(len(data_set)) if i not in cores_x]
                
                shuffle(rand_x)
                
                data_anti = return_anti(idx, rand_x, 10)
                
                for data_res in data_anti:
                    response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
                
            else:
                
                shuffle(cores_x)
                
                data_anti = return_anti(idx, cores_x, 5)
                
                for data_res in data_anti:
                    response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
                    
                rand_x = [i for i in range(len(data_set)) if i not in cores_x]
                
                shuffle(rand_x)
                
                data_anti = return_anti(idx, rand_x, 5)
                
                for data_res in data_anti:
                    response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
                    
    else:
        
        rand_x = [i for i in range(len(data_set))]
    
        shuffle(rand_x)
    
        data_anti = return_anti(idx, rand_x, 10)
    
        for data_res in data_anti:
            response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
        
    for idx2 in range(num_res):
        label.append(1)
        
    for idx2 in range(10):
        label.append(0)
    
    cont_tmp = [find_index(x, dic) for x in tokenize2(data_set[idx][0].strip())]
    for idx2 in range(num_res+10):
        context.append(cont_tmp)
    
    #print(cnt)
    cnt = cnt + 1

print(cnt)

train_set['c'] = context
train_set['r'] = response
train_set['y'] = label


78219


In [35]:
valid_set = {}
context = []
response = []
label = []

log_context = []
log_response = []

cnt = 0
for idx in range(len(data_set_val)):
    
    num_res = 0
    
    for data_res in data_set_val[idx][1]:
        response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
        log_response.append(data_res.strip())
        num_res = num_res + 1
    
    rand_x = [i for i in range(len(data_set_val))]
    shuffle(rand_x)
    
    data_anti = return_anti2(idx, rand_x)
    
    for data_res in data_anti:
        response.append( [find_index(x, dic) for x in tokenize2(data_res.strip())] )
        log_response.append(data_res.strip())
        
    for idx2 in range(num_res):
        label.append(1)
        
    for idx2 in range(10):
        label.append(0)
    
    cont_tmp = [find_index(x, dic) for x in tokenize2(data_set_val[idx][0].strip())]
    for idx2 in range(num_res+10):
        context.append(cont_tmp)
        log_context.append(data_set_val[idx][0].strip())
    
    #print(idx)
    cnt = cnt + 1

print(cnt)

valid_set['c'] = context
valid_set['r'] = response
valid_set['y'] = label


964


In [36]:
print(len(log_context))
print(len(log_response))


15414
15414


In [37]:
import csv
with open('log_val.csv', 'w', encoding='utf-8', newline='') as ft:
    wr = csv.writer(ft)
    for idx in range(len(log_context)):
        filtered_context = log_context[idx].replace('\n', '')
        filtered_response = log_response[idx].replace('\n', '')
        wr.writerow([filtered_context, filtered_response])


In [38]:
print(data_anti)
print(len(data_anti))

['임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?', '1989년 6월 30일 평양축전에 대표로 파견 된 인물은?', '임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 연도는?', '임종석을 검거한 장소는 경희대 내 어디인가?', '임종석이 조사를 받은 뒤 인계된 곳은 어딘가?', '1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의로 지명수배된 사람의 이름은?', '임종석이 1989년 2월 15일에 지명수배 받은 혐의는 어떤 시위를 주도했다는 것인가?', '제칠일안식교에서 비롯된 의사과학의 한 종류인 유사지질학의 이름은 무엇인가?', '극보수주의계열의 기독교이자 아직도 노아의 홍수가 있었다고 주장하는 곳은 어디인가?', '고대사회에서 성경은 교리를 다루는 책일 뿐만 아니라 어떤 책으로도 권위가 상당했는가?']
10


In [39]:
print(len(train_set['c']))
print(len(train_set['r']))
print(len(train_set['y']))

942865
942865
942865


In [40]:
print(len(valid_set['c']))
print(len(valid_set['r']))
print(len(valid_set['y']))

15414
15414
15414


In [41]:
pickle.dump( [train_set, valid_set], open('./v1_data.pkl', 'wb') )
